### Import Relevant Libraries 

In [1]:
import numpy as np
import pandas as pd
import keras as k
import math
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from PIL import Image

Using TensorFlow backend.


### Process Dataset

In [2]:
# Load MNIST dataset
MNIST = k.datasets.mnist.load_data(path="mnist.npz")
training = MNIST[0]
testing = MNIST[1]
X = training[0]
X_test = testing[0]
Y = to_categorical(training[1])
Y_test = to_categorical(testing[1])
# Resize all MNIST images to be 224x224, same dimensions as Arun's architecture
X = np.array([np.array(Image.fromarray(arr).resize([224,224])) for arr in X])
X_test = np.array([np.array(Image.fromarray(arr).resize([224,224])) for arr in X_test])
# Reshape array to be rank 1
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

### Define and compile model

In [70]:
model = k.Sequential([
    k.layers.InputLayer(input_shape=(224,224,1)),
    k.layers.Conv2D(filters=16, kernel_size=3),
    k.layers.MaxPool2D(pool_size=(2,2)),
    k.layers.Conv2D(filters=32, kernel_size=3),
    k.layers.MaxPool2D(pool_size=(2,2)),
    k.layers.Conv2D(filters=32, kernel_size=3),
    k.layers.Flatten(),
    k.layers.Dense(32, activation="relu"),
    k.layers.Dense(32, activation="relu"),
    k.layers.Dense(10, activation="softmax")
])

In [71]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 222, 222, 16)      160       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 52, 52, 32)        9248      
_________________________________________________________________
flatten_4 (Flatten)          (None, 86528)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 32)                2768928   
__________

In [72]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy')

In [62]:
# Train model
history = model.fit(x=X, y=Y, batch_size=200, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 29s 489us/step - loss: 0.9633
Epoch 2/5
60000/60000 [==============================] - 28s 474us/step - loss: 0.2602
Epoch 3/5
60000/60000 [==============================] - 29s 476us/step - loss: 0.1657
Epoch 4/5
60000/60000 [==============================] - 29s 477us/step - loss: 0.1219
Epoch 5/5
60000/60000 [==============================] - 29s 477us/step - loss: 0.0925


In [63]:
predictions = model.predict_classes(X_test)
accuracy_score(predictions, testing[1])

0.96

### Extract and process intermediate layer output

In [64]:
layer_name = 'flatten_3'
intermediate_layer_model = k.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_test)
intermediate_output.shape

In [ ]:
# Compute distance matrix
# TODO: Vectorize this for increased efficiency
distance = np.zeros((len(intermediate_output),len(intermediate_output)))
for i,vector1 in enumerate(intermediate_output):
    for j,vector2 in enumerate(intermediate_output):
        distance[i,j] = np.sqrt(np.sum(np.square(vector1-vector2)))

In [76]:
pd.DataFrame(distance)

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.000000,2143.550537,1757.277344,2069.921631,1849.526489,1730.397217,1789.560791,1916.627930,2089.604004,1803.953979,...,1633.446777,2308.438965,1864.345947,2332.670410,1820.548218,2165.089600,2034.295532,1895.737549,1857.516846,2251.586426
1,2143.550537,0.000000,1893.148438,2220.928711,2259.830811,1968.135132,2222.696777,2117.679932,2205.088135,2289.094238,...,2181.114746,2284.942139,2285.035645,2373.286865,1835.098145,2252.639893,2091.733887,2218.999268,2040.988159,2302.403809
2,1757.277344,1893.148438,0.000000,2034.541382,1911.256470,995.091858,1879.263916,1740.951538,1899.250732,1841.004639,...,1689.003662,2217.001953,1954.588135,2228.265381,1140.138550,2057.665039,1914.985474,1785.613159,1777.311279,2076.361084
3,2069.921631,2220.928711,2034.541382,0.000000,2121.737305,2049.062500,2243.940186,2085.571533,2184.530518,2176.483643,...,2145.248291,2359.600342,2186.236816,2188.281738,2031.834473,2337.326416,2070.173828,2114.762695,2094.004883,2073.208740
4,1849.526489,2259.830811,1911.256470,2121.737305,0.000000,1927.481812,1892.085693,1945.269653,1979.102661,2078.437988,...,2063.769043,2227.158447,1915.688965,2326.199463,1963.853149,2276.887939,2119.998535,2011.490234,2085.375732,2239.922852
5,1730.397217,1968.135132,995.091858,2049.062500,1927.481812,0.000000,1905.172974,1778.913452,1889.161743,1847.559326,...,1665.192505,2117.575195,1936.914062,2181.044922,1119.843506,2072.338867,1941.838867,1773.418579,1808.813965,2131.457520
6,1789.560791,2222.696777,1879.263916,2243.940186,1892.085693,1905.172974,0.000000,2035.265869,2109.710205,1989.868408,...,1920.285034,2297.090576,2028.122559,2394.531738,1953.571533,2177.179932,2099.436768,1946.289917,1999.075562,2337.172363
7,1916.627930,2117.679932,1740.951538,2085.571533,1945.269653,1778.913452,2035.265869,0.000000,2057.203857,2023.604858,...,1962.841431,2198.027100,2029.136230,2300.251709,1794.848267,2310.748047,2074.745117,1855.785645,2018.036377,2206.279785
8,2089.604004,2205.088135,1899.250732,2184.530518,1979.102661,1889.161743,2109.710205,2057.203857,0.000000,2189.168457,...,2041.940430,2338.409912,2060.791992,2253.911377,1931.180054,2307.062988,2170.113525,2135.817627,2156.530029,2207.700439
9,1803.953979,2289.094238,1841.004639,2176.483643,2078.437988,1847.559326,1989.868408,2023.604858,2189.168457,0.000000,...,1910.354370,2389.645996,1934.978760,2325.747070,1929.695435,2340.487549,2234.419678,1712.744995,1971.197510,2289.070557
